In [ ]:
from ceda_sentinel.s2_ard_links import find_image_links
from ceda_sentinel.s2_ard_read_write import *

- Input a date range and a polygon for area(s) of interest
- Find folders that are within that date range
- Find images that intersect the polygons
- Clip the images that intersect by the polygons and save geotiff of the interecting area

In [ ]:
# The root url where CEDA S2 ARD images are found
base_url = "https://data.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2"

In [ ]:
# The start date and end date over which to search for images
# Longer date range means more chance of cloud-free coverage, but also means longer spent searching
start_date = "2023-05-01"
end_date = "2023-05-31"

In [ ]:
# Read area of interest shapefile. In this case in the repo inputs folder
aoi_gdf = gpd.read_file("inputs/test_quarries.shp")

In [ ]:
# Process the AOI polygons:
# 1. Find a list of S2 image tile names that intersect the AOI
# 2. Find all CEDA S2 XML metadata file links within the date range containing those tile names.
# 3. Read each XML and if not too cloud extract image extent.
# 4. Spatial join image extent to AOI polygons. If more than one matching image, multiple rows are created.
aoi_image_gdf = find_image_links(aoi_gdf, start_date, end_date, base_url)

In [ ]:
# Plot one of the images for one AOI
plot_sample_image(aoi_image_gdf)

In [ ]:
# Write all window images to disk, by default they will be written to the outputs folder in this repo
write_s2_windows_to_tif(aoi_image_gdf)